In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

### Исходные данные

In [2]:
dates = pd.read_csv('Dates.csv', sep=';', encoding='ISO-8859-1')

dates.Date = dates.Date.map(lambda x: pd.to_datetime(x, format='%Y-%m-%d')).values
dates.weekday = dates.Date.map(lambda x: x.weekday()).values
dates = dates[dates.Date >= '2018-01-01'].reset_index(drop=True)

Файл 'Dates.csv' содержит информацию за период 2010-2020 гг., включая:

    - "number" - порядковый номер дня в году;
    - "weekday" - день недели (0 - понедельник, ..., 6 - воскресенье);
    - "half_holiday" - индикатор сокращенного рабочего дня;
    - "All_weekends" - индикатор выходного дня (в т.ч. празничного выходного дня);
    - "Holiday", "Holiday_weekends", "Regular_weekends" - индикаторы празника, праздничного и обычного выходного дня
    
Признаки составлены на основе производственного календаря:
https://calendar.yoip.ru/work/2020-proizvodstvennyj-calendar.html

In [3]:
dates.head()

,Date,number,weekday,half_holiday,All_weekends,Holiday,Holiday_weekends,Regular_weekends
0,2018-01-01,1,0,0,1,1,1,0
1,2018-01-02,2,1,0,1,0,1,0
2,2018-01-03,3,2,0,1,0,1,0
3,2018-01-04,4,3,0,1,0,1,0
4,2018-01-05,5,4,0,1,0,1,0


In [4]:
daylight_hours = pd.read_csv('световой день.csv', sep=';')

daylight_hours.Sunrise = daylight_hours.Date + ' ' + daylight_hours.Sunrise
daylight_hours.Sunset = daylight_hours.Date + ' ' + daylight_hours.Sunset
daylight_hours.Date = daylight_hours.Date.map(lambda x: pd.to_datetime(x, format='%d.%m.%Y')).values
daylight_hours.Sunrise = daylight_hours.Sunrise.map(lambda x: pd.to_datetime(x, format='%d.%m.%Y %H:%M:%S')).values
daylight_hours.Sunset = daylight_hours.Sunset.map(lambda x: pd.to_datetime(x, format='%d.%m.%Y %H:%M:%S')).values

Файл 'световой день.csv' содержит информацию за период 2018-2020 гг., включая:

    - "Sunrise" - время восхода солнца;
    - "Sunset" - время захода солнца
    
Информация о времени восхода и захода солнца в Москве взята с сайта:
https://flymeteo.org/other/astro/astro.php

In [5]:
daylight_hours.head()

,Date,Sunrise,Sunset
0,2017-12-31,2017-12-31 09:01:00,2017-12-31 16:06:00
1,2018-01-01,2018-01-01 09:00:00,2018-01-01 16:07:00
2,2018-01-02,2018-01-02 08:59:00,2018-01-02 16:08:00
3,2018-01-03,2018-01-03 08:59:00,2018-01-03 16:10:00
4,2018-01-04,2018-01-04 08:59:00,2018-01-04 16:11:00


In [6]:
weather = pd.read_csv('weather.csv', sep=';', decimal=',')
weather.columns = ['time', 'T', 'P0', 'P', 'U', 'DD', 'Ff', 'ff10', 'WW', "W'W'", 'c', 'VV', 'Td']
weather.time = weather.time.map(lambda x: pd.to_datetime(x, format='%d.%m.%Y %H:%M')).values
weather.sort_values('time', inplace=True)
weather.reset_index(drop=True, inplace=True)
weather = weather[['time', 'T', 'U', 'c','VV']] \
    .rename({'time' : 'Datetime', 'T' : 'temperature', 'U' : 'humidity', 'c' : 'cloudy', 'VV' : 'visibility'}, axis=1)

def fun(x):
    lst = ['continuous', 'Torn', 'Scattered', 'light', 'No', 'Vertical_visibility', "НЗ"]
    lst1 = ['плошная', 'азорванная', 'ассеянная', 'езначительная', 'Нет', 'Вертикальная видимость', 'НЗ']
    for i in range(len(lst)):
        if lst1[i] in x:
            return lst[i]
weather.cloudy = weather.cloudy.map(fun)
weather.loc[weather.cloudy=='НЗ', 'cloudy'] = None

weather['Date'] = weather.Datetime.map(lambda x: pd.to_datetime(x.date())).values

Файл 'weather.csv' содержит информацию о погоде за период 2018-2020 гг., включая:

    - "temperature" - температура воздуха;
    - "humidity" - относительная влажность воздуха в процентах;
    - "cloudy" - тип облачности;
    - "visibility" - горизонтальная видимость
    
Архив погоды до 25.11.2020 и прогноз с 26.11.2020 до 01.12.2020 взят с сайта: 
           https://rp5.ru/%D0%90%D1%80%D1%85%D0%B8%D0%B2_%D0%BF%D0%BE%D0%B3%D0%BE%D0%B4%D1%8B_%D0%B2_%D0%A8%D0%B5%D1%80%D0%B5%D0%BC%D0%B5%D1%82%D1%8C%D0%B5%D0%B2%D0%BE,_%D0%B8%D0%BC._%D0%90._%D0%A1._%D0%9F%D1%83%D1%88%D0%BA%D0%B8%D0%BD%D0%B0_(%D0%B0%D1%8D%D1%80%D0%BE%D0%BF%D0%BE%D1%80%D1%82),_METAR

Выгрузка файла - в формате xls, после - сохраняем в csv. В колонке VV заменяем "10.0 и более" на "10" и сортируем все по дате.
В колонке "с" отсутствующие значения помечаем как "НЗ".
(заполнение прогнозов погоды - вручную).

Прогноз погоды с 01.12.2020 до 04.12.2020 взят с сайта:
https://www.gismeteo.ru/weather-moscow-sheremetyevo-13068/month/

In [7]:
weather.head()

,Datetime,temperature,humidity,cloudy,visibility,Date
0,2018-01-01 00:00:00,0.0,100.0,Torn,9.0,2018-01-01
1,2018-01-01 00:30:00,0.0,100.0,Torn,9.0,2018-01-01
2,2018-01-01 01:00:00,0.0,100.0,Torn,8.0,2018-01-01
3,2018-01-01 01:30:00,0.0,100.0,Torn,4.3,2018-01-01
4,2018-01-01 02:00:00,0.0,93.0,continuous,8.0,2018-01-01


In [8]:
covid = pd.read_csv('Ковид.csv', sep=';')
covid.Date = covid.Date.map(lambda x: pd.to_datetime(x, format='%d.%m.%Y')).values

Файл 'Ковид.csv' содержит признак "COVID_isolation" - индикатор периода самоизоляции (за 2020 год).

Периоды самоизоляции выделены согласно указам мэра Москвы:
https://www.mos.ru/city/projects/covid-19/documents/

In [9]:
covid.head()

,Date,COVID_isolation
0,2020-01-01,0
1,2020-01-02,0
2,2020-01-03,0
3,2020-01-04,0
4,2020-01-05,0


### Создание дополнительных переменных

In [10]:
base = pd.to_datetime('2018-01-01 00:00:00', format='%Y-%m-%d %H:%M:%S')
date_list = [base]
while base != pd.to_datetime('2020-12-31 23:00:00', format='%Y-%m-%d %H:%M:%S'):
    base += timedelta(hours=1)
    date_list.append(base)

In [11]:
features = pd.DataFrame({'Datetime' : date_list})

features['Date'] = features.Datetime.map(lambda x: pd.to_datetime(x.date())).values

features = features.merge(daylight_hours, how='left')
features = features.merge(dates, how='left')
features = features.merge(weather, how='left')
features = features.merge(covid, how='left')
features.COVID_isolation.fillna(0, inplace=True)

# порядковый день в периоде самоизоляции

data = features.loc[features.COVID_isolation == 1, ['Date', 'COVID_isolation']].drop_duplicates().sort_values('Date')
data['delta_days'] = data.Date.diff()
data = data.fillna(pd.Timedelta(days=2))

def fun(x):
    a = 0
    lst = []
    for i in range(len(x)):
        if x[i] > pd.Timedelta(days=1):
            a = 0
            lst.append(a)
        else:
            a += 1
            lst.append(a)
    return lst

    
data['day_in_isolation'] = fun(list(data.delta_days))
data = data.drop(['delta_days', 'COVID_isolation'], axis=1)

features = features.merge(data, how='left')
features.day_in_isolation.fillna(0, inplace=True)

# длительность светового дня

features['daylight_hours'] = features.Sunset - features.Sunrise

# следующий рассвет

features['Date_plus1'] = (features['Date'] + timedelta(days=1)).values
df = (features[['Date']] + timedelta(days=1)).merge(daylight_hours[['Date', 'Sunrise']]) \
        .rename({'Sunrise' : 'Next_sunrise', 'Date' : 'Date_plus1'}, axis=1).drop_duplicates()
features = features.merge(df, how='left')
features = features.drop('Date_plus1', axis=1)

# предыдущий рассвет

features['Date_minus1'] = (features['Date'] - timedelta(days=1)).values
df = (features[['Date']] - timedelta(days=1)).merge(daylight_hours[['Date', 'Sunrise']]) \
.rename({'Sunrise' : 'Previous_sunrise', 'Date' : 'Date_minus1'}, axis=1).drop_duplicates()
features = features.merge(df, how='left')
features = features.drop('Date_minus1', axis=1)

# следующий закат

features['Date_plus1'] = (features['Date'] + timedelta(days=1)).values
df = (features[['Date']] + timedelta(days=1)).merge(daylight_hours[['Date', 'Sunset']]) \
        .rename({'Sunset' : 'Next_sunset', 'Date' : 'Date_plus1'}, axis=1).drop_duplicates()
features = features.merge(df, how='left')
features = features.drop('Date_plus1', axis=1)

# время с момента последнего рассвета

features.loc[features.Datetime > features.Sunrise, 'From_last_sunrise'] = (features.Datetime - features.Sunrise)
features.loc[features.Datetime <= features.Sunrise, 'From_last_sunrise'] = (features.Datetime - features.Previous_sunrise)

# время до следующего заката

features.loc[features.Datetime > features.Sunset, 'Until_next_sunset'] = (features.Next_sunset - features.Datetime)
features.loc[features.Datetime <= features.Sunset, 'Until_next_sunset'] = (features.Sunset - features.Datetime)

# индикатор светлого времени суток

features.loc[(features.Sunrise > features.Datetime) | (features.Datetime >= features.Sunset), 'Is_light'] = 0
features.loc[(features.Sunrise <= features.Datetime) & (features.Datetime < features.Sunset), 'Is_light'] = 1

# индикатор предпраздничного дня

df = features.loc[features.Holiday_weekends == 1, ["Date"]].drop_duplicates()
df['delta_days'] = df.Date.diff()
df = df.fillna(pd.Timedelta(days=2))
features['pre_holiday_day'] = features.Date \
    .map(lambda x: 1 if x in list(df.loc[df.delta_days > pd.Timedelta(days=1), 'Date'] - timedelta(days=1)) else 0)

# час внутри дня

features['hour'] = features.Datetime.map(lambda x: x.hour)

# месяц

features['month'] = features.Datetime.map(lambda x: x.month)

# день в месяце

features['day_in_month'] = features.Datetime.map(lambda x: x.day)

# индикатор рабочего времени

features['work_time'] = features.hour.map(lambda x: 1 if 9 <= x <= 18 else 0)

# час с начала рабочего дня

df = features.loc[features.work_time==1, ['hour']].drop_duplicates().sort_values('hour')
df['hour_since_start_work'] = [i for i in range(10)]
features = features.merge(df, how='left')
features.hour_since_start_work.fillna(0, inplace=True)

# час с конца рабочего дня

df = features.loc[features.work_time==0, ['hour']].drop_duplicates().sort_values('hour')
df['hour_since_end_work'] = [i for i in range(6, 15)] + [i for i in range(1, 6)]
features = features.merge(df, how='left')
features.hour_since_end_work.fillna(0, inplace=True)


# преобразование признаков светового дня

features.daylight_hours = features.daylight_hours.map(lambda x: x.total_seconds()/3600).values
features.From_last_sunrise = features.From_last_sunrise.map(lambda x: x.total_seconds()/3600).values
features.Until_next_sunset = features.Until_next_sunset.map(lambda x: x.total_seconds()/3600).values

In [12]:
features = features[['Datetime', 'month', 'day_in_month', 'hour', 'number', 'weekday',
                     'half_holiday', 'All_weekends', 'Holiday', 'pre_holiday_day', 'Holiday_weekends', 'Regular_weekends',
                     'daylight_hours', 'From_last_sunrise', 'Until_next_sunset', 'Is_light',
                     'work_time', 'hour_since_start_work', 'hour_since_end_work',
                     'temperature', 'humidity', 'cloudy', 'visibility',
                     'COVID_isolation', 'day_in_isolation']]

### Восстановление пропущенных значений показателей погодных условий

In [13]:
df = features[['temperature', 'humidity', 'cloudy', 'visibility']]
features = features.drop(['temperature', 'humidity', 'cloudy', 'visibility'], axis=1)

# восстановление температуры воздуха

rf = RandomForestRegressor(n_estimators=500, max_depth=5, max_features=0.7, max_samples=0.8)
df_train = pd.concat([df[['temperature']], features], axis=1)
df_train.dropna(inplace=True)
rf.fit(df_train.drop(['temperature', 'Datetime',
                      'COVID_isolation', 'day_in_isolation',
                      'work_time', 'hour_since_start_work',
                      'hour_since_end_work','weekday',
                      'half_holiday', 'All_weekends',
                      'Holiday', 'pre_holiday_day',
                      'Holiday_weekends', 'Regular_weekends'],axis=1), df_train.temperature)
features =  pd.concat([df[['temperature']], features], axis=1)
features.loc[df.temperature.isna(), 'temperature'] = \
rf.predict(features.drop(['temperature', 'Datetime',
                          'COVID_isolation', 'day_in_isolation',
                          'work_time', 'hour_since_start_work',
                          'hour_since_end_work','weekday',
                          'half_holiday', 'All_weekends',
                          'Holiday', 'pre_holiday_day',
                          'Holiday_weekends', 'Regular_weekends'], axis=1)[df.temperature.isna()])

# восстановление относительной влажности воздуха

df_train = pd.concat([df[['humidity']], features], axis=1)
df_train.dropna(inplace=True)
rf.fit(df_train.drop(['humidity', 'Datetime',
                      'COVID_isolation', 'day_in_isolation',
                      'work_time', 'hour_since_start_work',
                      'hour_since_end_work','weekday',
                      'half_holiday', 'All_weekends',
                      'Holiday', 'pre_holiday_day',
                      'Holiday_weekends', 'Regular_weekends'],axis=1), df_train.humidity)
features =  pd.concat([df[['humidity']], features], axis=1)
features.loc[df.humidity.isna(), 'humidity'] = \
rf.predict(features.drop(['humidity', 'Datetime',
                          'COVID_isolation', 'day_in_isolation',
                          'work_time', 'hour_since_start_work',
                          'hour_since_end_work','weekday',
                          'half_holiday', 'All_weekends',
                          'Holiday', 'pre_holiday_day',
                          'Holiday_weekends', 'Regular_weekends'],axis=1)[df.humidity.isna()])

# восстановление горизонтальной видимости

df_train = pd.concat([df[['visibility']], features], axis=1)
df_train.dropna(inplace=True)
rf.fit(df_train.drop(['visibility', 'Datetime',
                      'COVID_isolation', 'day_in_isolation',
                      'work_time', 'hour_since_start_work',
                      'hour_since_end_work','weekday',
                      'half_holiday', 'All_weekends',
                      'Holiday', 'pre_holiday_day',
                      'Holiday_weekends', 'Regular_weekends'],axis=1), df_train.visibility)
features =  pd.concat([df[['visibility']], features], axis=1)
features.loc[df.visibility.isna(), 'visibility'] = \
rf.predict(features.drop(['visibility', 'Datetime',
                          'COVID_isolation', 'day_in_isolation',
                          'work_time', 'hour_since_start_work',
                          'hour_since_end_work','weekday',
                          'half_holiday', 'All_weekends',
                          'Holiday', 'pre_holiday_day',
                          'Holiday_weekends', 'Regular_weekends'],axis=1)[df.visibility.isna()])

# восстановление типа облачности

rf = RandomForestClassifier(n_estimators=500, max_depth=5, max_features=0.7, max_samples=0.8)
df_train = pd.concat([df[['cloudy']], features], axis=1)
df_train.dropna(inplace=True)
rf.fit(df_train.drop(['cloudy', 'Datetime',
                      'COVID_isolation', 'day_in_isolation',
                      'work_time', 'hour_since_start_work',
                      'hour_since_end_work','weekday',
                      'half_holiday', 'All_weekends',
                      'Holiday', 'pre_holiday_day',
                      'Holiday_weekends', 'Regular_weekends'],axis=1), df_train.cloudy)
features =  pd.concat([df[['cloudy']], features], axis=1)
features.loc[df.cloudy.isna(), 'cloudy'] = \
rf.predict(features.drop(['cloudy', 'Datetime',
                          'COVID_isolation', 'day_in_isolation',
                          'work_time', 'hour_since_start_work',
                          'hour_since_end_work','weekday',
                          'half_holiday', 'All_weekends',
                          'Holiday', 'pre_holiday_day',
                          'Holiday_weekends', 'Regular_weekends'],axis=1)[df.cloudy.isna()])

features = features[['Datetime', 'month', 'day_in_month', 'hour', 'number', 'weekday',
                     'half_holiday', 'All_weekends', 'Holiday', 'pre_holiday_day', 'Holiday_weekends', 'Regular_weekends',
                     'daylight_hours', 'From_last_sunrise', 'Until_next_sunset', 'Is_light',
                     'work_time', 'hour_since_start_work', 'hour_since_end_work',
                     'temperature', 'humidity', 'cloudy', 'visibility',
                     'COVID_isolation', 'day_in_isolation']]


# перевод времени в UTC

features.Datetime = (features.Datetime - timedelta(hours=3)).values

In [14]:
# features.to_csv('data_UTC.csv', index=False)

Итоговый файл data_UTC.csv содержит информацию за период 2018-2020 гг. по следующим группам признаков:

    1) Дата и время:
        - "month" - номер месяца (категориальная переменная);
        - "day_in_month" - порядковый день месяца (количественная переменная);
        - "number" - порядковый день в году (количественная переменная);
        - "weekday" - день недели (категориальная переменная);
        - "hour" - час (категориальная переменная).
        
    2) Тип дня (индикаторные переменные):
        - "Holiday" - праздники;
        - "pre_holiday_day" - предпраздничные дни;
        - "half_holiday" - сокращенные рабочие дни;
        - "Holiday_weekends" - праздничные выходные;
        - "Regular_weekends" - обычные выходные дни;
        - "All_weekends" - все выходные дни.
    
    3) Световой день:
        - "daylight_hours" - длительность световогодня в часах (количественная переменная);
        - "From_last_sunrise" - время с момента последнего рассвета в часах (количественная переменная);
        - "Until_next_sunset" - время до момента следующего заката в часах (количественная переменная);
        - "Is_light" - индикатор светлого времени суток.
        
    4) Внутридневные часы:
        - "work_time" - индикатор рабочего времени;
        - "hour_since_start_work" - час с момента начала (9:00 мск) рабочего времени (количественная переменная);
        - "hour_since_end_work" - час с момента конца (18:00 мск) рабочего времени (количественная переменная);
     
    5) Погодные условия:
        - "temperature" - температура в градусах Цельсия (количественная переменная);
        - "humidity"  - относительная влажность в процентах (количественная переменная);
        - "cloudy" - тип облачности (категориальная переменная);
        - "visibility" - горизонтальная видимость (количественная переменная).
        
        Прогноз погоды с 17.12.2020 по конец года составлен с помощью последовательного построения моделей random forest на данных за весь прошлый период с 01.01.2018 года.
        
    6) COVID:
        - "COVID_isolation" - индикатор периода самоизоляции;
        - "day_in_isolation" - порядковый день внутри периода самоизоляции (количественная переменная).

In [16]:
features.head(10)

,Datetime,month,day_in_month,hour,number,weekday,half_holiday,All_weekends,Holiday,pre_holiday_day,...,Is_light,work_time,hour_since_start_work,hour_since_end_work,temperature,humidity,cloudy,visibility,COVID_isolation,day_in_isolation
0,2017-12-31 21:00:00,1,1,0,1,0,0,1,1,0,...,0.0,0,0.0,6.0,0.0,100.0,Torn,9.0,0.0,0.0
1,2017-12-31 22:00:00,1,1,1,1,0,0,1,1,0,...,0.0,0,0.0,7.0,0.0,100.0,Torn,8.0,0.0,0.0
2,2017-12-31 23:00:00,1,1,2,1,0,0,1,1,0,...,0.0,0,0.0,8.0,0.0,93.0,continuous,8.0,0.0,0.0
3,2018-01-01 00:00:00,1,1,3,1,0,0,1,1,0,...,0.0,0,0.0,9.0,0.0,93.0,continuous,10.0,0.0,0.0
4,2018-01-01 01:00:00,1,1,4,1,0,0,1,1,0,...,0.0,0,0.0,10.0,0.0,87.0,continuous,10.0,0.0,0.0
5,2018-01-01 02:00:00,1,1,5,1,0,0,1,1,0,...,0.0,0,0.0,11.0,0.0,93.0,continuous,10.0,0.0,0.0
6,2018-01-01 03:00:00,1,1,6,1,0,0,1,1,0,...,0.0,0,0.0,12.0,0.0,93.0,continuous,10.0,0.0,0.0
7,2018-01-01 04:00:00,1,1,7,1,0,0,1,1,0,...,0.0,0,0.0,13.0,0.0,87.0,continuous,10.0,0.0,0.0
8,2018-01-01 05:00:00,1,1,8,1,0,0,1,1,0,...,0.0,0,0.0,14.0,0.0,93.0,continuous,10.0,0.0,0.0
9,2018-01-01 06:00:00,1,1,9,1,0,0,1,1,0,...,1.0,1,0.0,0.0,0.0,93.0,continuous,10.0,0.0,0.0


In [15]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26304 entries, 0 to 26303
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Datetime               26304 non-null  datetime64[ns]
 1   month                  26304 non-null  int64         
 2   day_in_month           26304 non-null  int64         
 3   hour                   26304 non-null  int64         
 4   number                 26304 non-null  int64         
 5   weekday                26304 non-null  int64         
 6   half_holiday           26304 non-null  int64         
 7   All_weekends           26304 non-null  int64         
 8   Holiday                26304 non-null  int64         
 9   pre_holiday_day        26304 non-null  int64         
 10  Holiday_weekends       26304 non-null  int64         
 11  Regular_weekends       26304 non-null  int64         
 12  daylight_hours         26304 non-null  float64       
 13  F